TAREA: Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc. 

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

Importación de paquetes

In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

Lectura de imágenes y variables globales

In [3]:
# Lectura de imágenes
imagen = cv2.imread("Monedas.jpg")
imagen_p1 = cv2.imread("VC - Prueba-Monedas1.jpg")
imagen_p2 = cv2.imread("VC - Prueba-Monedas2.jpg")

img = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
img_rgb_p1 = cv2.cvtColor(imagen_p1, cv2.COLOR_BGR2RGB)
img_rgb_p2 = cv2.cvtColor(imagen_p2, cv2.COLOR_BGR2RGB)

# Variables globales
coins = [0] * 8
circulos_filtrados = []
diametros_reales = {
    0.01: 16.25,  # Moneda de 1 céntimo
    0.02: 18.75,  # Moneda de 2 céntimos
    0.05: 21.25,  # Moneda de 5 céntimos
    0.1: 19.75,   # Moneda de 10 céntimos
    0.2: 22.25,   # Moneda de 20 céntimos
    0.5: 24.25,   # Moneda de 50 céntimos (referencia)
    1.0: 23.25,   # Moneda de 1 euro
    2.0: 25.75    # Moneda de 2 euros
}


Función que cambia el fondo a blanco.

In [4]:
def convertir_fondo_a_blanco(imagen):
     # Convertir la imagen a espacio de color HSV
    # Convertir la imagen a espacio de color HSV para crear una máscara
    img_hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de color para detectar monedas en HSV
    lower_bound = np.array([0, 30, 40])    # Límite inferior
    upper_bound = np.array([50, 255, 255]) # Límite superior
    
    # Crear una máscara que selecciona solo las monedas
    mascara = cv2.inRange(img_hsv, lower_bound, upper_bound)
    
    # Realizar una operación de cierre morfológico para unir áreas y mejorar la detección de monedas
    kernel = np.ones((5, 5), np.uint8)
    mascara_cerrada = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel)
    
    # Detectar contornos en la máscara
    contornos, _ = cv2.findContours(mascara_cerrada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Crear una máscara vacía en la que solo conservaremos los contornos grandes (monedas)
    mascara_filtrada = np.zeros_like(mascara_cerrada)
    
    # Filtrar contornos por área, manteniendo solo los suficientemente grandes para ser monedas
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        
        # Filtrar restos pequeños: ajustar el área mínima según sea necesario
        if area > 1000:  # Mantener solo los contornos grandes (monedas)
            cv2.drawContours(mascara_filtrada, [contorno], -1, 255, -1)  # Rellenar el contorno en la máscara filtrada

    # Aplicar la máscara filtrada sobre la imagen original para cambiar el fondo a blanco
    imagen_blanco_fondo = imagen.copy()
    imagen_blanco_fondo[mascara_filtrada == 0] = [255, 255, 255]  # Asignar blanco a las áreas fuera de las monedas

    return imagen_blanco_fondo

Función que detecta círculos dentro de la imagen.

In [5]:
def detectar_circulos(imagen):
    # Convertir la imagen a escala de grises
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Aplicar desenfoque para reducir el ruido y mejorar la detección de bordes
    imagen_gris = cv2.medianBlur(imagen_gris, 5)
    
    # Detectar círculos con HoughCircles
    circulos = cv2.HoughCircles(imagen_gris, 
                                cv2.HOUGH_GRADIENT, 
                                dp=1.01, 
                                minDist=150, 
                                param1=100, 
                                param2=25, 
                                minRadius=15, 
                                maxRadius=110)
    
    # Lista para almacenar los círculos detectados
    circulos_detectados = []
    
    if circulos is not None:
        # Redondear y convertir los círculos detectados a enteros
        circulos = np.round(circulos[0, :]).astype("int")
        
        # Guardar cada círculo en la lista
        for (x, y, r) in circulos:
            circulos_detectados.append((x, y, r))
    
    return circulos_detectados

Función que dibuja los círculos en una imagen negra.

In [6]:
def imagen_blanca_negra(img_rgb, circulos):
    #Dibuja contornos externos rellenos en imagen vacía
    #Imagen negra
    img_circulos = np.zeros((img_rgb.shape[0], img_rgb.shape[1]), dtype=np.uint8)
    
    # Dibujar cada círculo en la imagen negra
    for (x, y, r) in circulos:
        # Dibujar el círculo en blanco
        cv2.circle(img_circulos, (x, y), int(r), 255, thickness=-1)  # -1 rellena el círculo
    
    return img_circulos

Funcion para filtrar los circulos.

In [7]:
def filtrar_por_color(imagen, circulos, rango_color_min, rango_color_max):
    # Convertir la imagen a HSV
    # Convertir la imagen a HSV
    imagen_hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    circulos_filtrados = []

    for (x, y, r) in circulos:
        # Crear una máscara para el círculo actual
        mascara = np.zeros(imagen.shape[:2], dtype=np.uint8)
        cv2.circle(mascara, (x, y), r, 255, -1)

        # Calcular el color promedio dentro del círculo en HSV
        color_promedio = cv2.mean(imagen_hsv, mask=mascara)[:3]

        # Verificar si el color promedio está dentro del rango
        if (rango_color_min[0] <= color_promedio[0] <= rango_color_max[0] and
            rango_color_min[1] <= color_promedio[1] <= rango_color_max[1] and
            rango_color_min[2] <= color_promedio[2] <= rango_color_max[2]):
            # Si cumple con el rango de color, añadirlo a la lista filtrada
            circulos_filtrados.append((x, y, r))

    return circulos_filtrados

Función para superponer los círculos en la imagen original. 

In [8]:
def superponer_circulos(imagen, circulos):
    # Crear una copia de la imagen para no modificar la original
    imagen_superpuesta = imagen.copy()

    # Verificar que la lista de círculos no esté vacía
    if not circulos:
        print("No se detectaron círculos.")
        return imagen_superpuesta

    # Dibujar los círculos sobre la imagen
    for (x, y, r) in circulos:
        cv2.circle(imagen_superpuesta, (x, y), r, (0, 255, 0), 2)  # Círculo verde
        cv2.circle(imagen_superpuesta, (x, y), 2, (0, 0, 255), 3)  # Centro del círculo en rojo

    return imagen_superpuesta

Función para obtener radio de referencia.

In [9]:
def obtener_radio_referencia(circulos, punto_click):
    for (x, y, r) in circulos:
        distancia = np.sqrt((punto_click[0] - x) ** 2 + (punto_click[1] - y) ** 2)
        if distancia <= r:
            return r
    return None

Función para clasificar las monedas.

In [10]:
def clasificar_monedas(circulos, relacion_escala):

     # Reiniciar coins antes de clasificar
    monedas_clasificadas = [0] * 8

    for (x, y, r) in circulos:
        # Calcular el diámetro real de la moneda detectada en milímetros
        diametro_detectado = 2 * r  # Diámetro en píxeles
        diametro_real = diametro_detectado * relacion_escala

        # Encontrar la moneda cuyo diámetro real más se acerque al diámetro calculado
        diferencia_minima = float('inf')
        valor_asignado = None
        for valor, diametro in diametros_reales.items():
            diferencia = abs(diametro_real - diametro)
            if diferencia < diferencia_minima:
                diferencia_minima = diferencia
                valor_asignado = valor

        # Actualizar el conteo de monedas según el valor asignado
        if valor_asignado == 0.01:
            monedas_clasificadas[0] += 1
        elif valor_asignado == 0.02:
            monedas_clasificadas[1] += 1
        elif valor_asignado == 0.05:
            monedas_clasificadas[2] += 1
        elif valor_asignado == 0.1:
            monedas_clasificadas[3] += 1
        elif valor_asignado == 0.2:
            monedas_clasificadas[4] += 1
        elif valor_asignado == 0.5:
            monedas_clasificadas[5] += 1
        elif valor_asignado == 1.0:
            monedas_clasificadas[6] += 1
        elif valor_asignado == 2.0:
            monedas_clasificadas[7] += 1

    # Actualizar la lista global `coins` al finalizar la clasificación
    global coins
    coins = monedas_clasificadas

moneCalculate(), función que se encarga de calcular el dinero total de la imagen

In [11]:
def moneyCalculate():
    return coins[0]*0.01 + coins[1]*0.02 + coins[2]*0.05 + coins[3]*0.1 + coins[4]*0.2 + coins[5]*0.5 + coins[6]*1 + coins[7]*2

Función que muestra las monedas detectadas y la relación asociada

In [12]:
def mostrar_resultados_deteccion(coins, relacion_escala):
    valores_monedas = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0]
    nombres_monedas = [
        "Monedas de 1 céntimo",
        "Monedas de 2 céntimos",
        "Monedas de 5 céntimos",
        "Monedas de 10 céntimos",
        "Monedas de 20 céntimos",
        "Monedas de 50 céntimos",
        "Monedas de 1 euro",
        "Monedas de 2 euros"
    ]
    
    print("Resultados de la detección de monedas:")
    print("=====================================")
    for i, cantidad in enumerate(coins):
        if cantidad > 0:
            print(f"{nombres_monedas[i]}: {cantidad}")
    
    print("\nRelación de escala utilizada para la detección:")
    print(f"Relación de escala (mm/píxeles): {relacion_escala:.4f}")

# Ejemplo de uso de la función para mostrar resultados
# Supongamos que ya tenemos la relación de escala calculada y el conteo de monedas
relacion_escala_ejemplo = 0.8  # Relación de escala de ejemplo
coins_ejemplo = [3, 2, 1, 4, 2, 5, 0, 1]  # Ejemplo de cantidades de monedas detectadas

Función que muestra la imagen en una ventana.

In [13]:
def mostrar_imagen(imagen, ancho=1100, alto=600):
    def detectar_color_y_radio(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Obtener el radio de la moneda de referencia (50 céntimos)
            radio_referencia = obtener_radio_referencia(circulos_filtrados, (x, y))
            if radio_referencia is not None:
                print(f"Radio de la moneda de referencia (50 céntimos): {radio_referencia}")

                # Calcular el diámetro detectado y la relación de escala
                diametro_detectado = 2 * radio_referencia
                diametro_real_50c = diametros_reales[0.5]
                relacion_escala = diametro_real_50c / diametro_detectado
                print(f"Relación de escala (mm/píxeles): {relacion_escala}")

                # Clasificar las monedas detectadas usando la relación de escala
                clasificar_monedas(circulos_filtrados, relacion_escala)

                # Calcular el dinero total
                total_dinero = moneyCalculate()
                print(f"Total de dinero en la imagen: {total_dinero:.2f} euros")
                mostrar_resultados_deteccion(coins,relacion_escala)
            else:
                print("No se encontró una moneda de referencia en el punto seleccionado.")

    # Crear una ventana de tamaño fijo
    cv2.namedWindow("Imagen con Fondo Blanco", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Imagen con Fondo Blanco", ancho, alto)

    # Asignar el callback para el evento de clic
    cv2.setMouseCallback("Imagen con Fondo Blanco", detectar_color_y_radio)

    # Mostrar la imagen en la ventana
    cv2.imshow("Imagen con Fondo Blanco", imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Paso final: Resultados.

In [14]:
imagen_con_fondo_blanco = convertir_fondo_a_blanco(imagen_p2)
circulos_detectados  = detectar_circulos(imagen_con_fondo_blanco)
# Filtrado doradas
circulos_filtrados.extend(filtrar_por_color(imagen_con_fondo_blanco, 
                                            circulos_detectados, (20, 0, 0), (180, 170, 255)))
# Filtrado cobrizas
circulos_filtrados.extend(filtrar_por_color(imagen_con_fondo_blanco,
                                             circulos_detectados, (9, 79, 38), (19, 220, 255)))
imagen_con_circulos = superponer_circulos(imagen_p2, circulos_filtrados)

#imagen_extrapolada = imagen_blanca_negra(img_rgb_p1,circulos_filtrados)

mostrar_imagen(imagen_con_circulos)

Radio de la moneda de referencia (50 céntimos): 84
Relación de escala (mm/píxeles): 0.14434523809523808
Total de dinero en la imagen: 4.07 euros
Resultados de la detección de monedas:
Monedas de 1 céntimo: 3
Monedas de 2 céntimos: 2
Monedas de 5 céntimos: 4
Monedas de 10 céntimos: 1
Monedas de 20 céntimos: 1
Monedas de 50 céntimos: 1
Monedas de 1 euro: 1
Monedas de 2 euros: 1

Relación de escala utilizada para la detección:
Relación de escala (mm/píxeles): 0.1443
